# Setup 

In [3]:
%cd /dss/dsshome1/03/ge87wod2/morphological-inflection/ByT5/

/dss/dsshome1/03/ge87wod2/morphological-inflection/ByT5


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
!pip install transformers torch datasets pip accelerate
!pip install transformers --upgrade
!pip install ipywidgets
!pip install python-dotenv pickleshare

  Using cached transformers-4.57.1-py3-none-any.whl.metadata (43 kB)
  Using cached accelerate-1.11.0-py3-none-any.whl.metadata (19 kB)
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached huggingface_hub-0.36.0-py3-none-any.whl.metadata (14 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 1.1 MB/s eta 0:00:00
  Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached safetensors-0.6.2-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.8.93-py3-none-manylinux2010_x86_64.manylinux_2_12_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_runtime_cu12-12.8.90-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.7 kB)
  Using cached nvidia_cuda_cupti_cu12-12.8.90-py3-none-manylinux2

  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
Using cached transformers-4.57.1-py3-none-any.whl (12.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.8/899.8 MB 11.4 MB/s eta 0:00:0000:0100:01
Using cached nvidia_cublas_cu12-12.8.4.1-py3-none-manylinux_2_27_x86_64.whl (594.3 MB)
Using cached nvidia_cuda_cupti_cu12-12.8.90-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (10.2 MB)
Using cached nvidia_cuda_nvrtc_cu12-12.8.93-py3-none-manylinux2010_x86_64.manylinux_2_12_x86_64.whl (88.0 MB)
Using cached nvidia_cuda_runtime_cu12-12.8.90-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (954 kB)
Using cached nvidia_cudnn_cu12-9.10.2.21-py3-none-manylinux_2_27_x86_64.whl (706.8 MB)
Using cached nvidia_cufft_cu12-11.3.3.83-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (193.1 MB)
Using cached nvidia_cufile_cu12-1.13.1.3-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (1.2 MB)
Using cached nvidia_curand_cu12-10.3.9.90-py3-none-manylinux_2_27_

  Using cached widgetsnbextension-4.0.15-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.16-py3-none-any.whl.metadata (20 kB)
Using cached ipywidgets-8.1.8-py3-none-any.whl (139 kB)
Using cached jupyterlab_widgets-3.0.16-py3-none-any.whl (914 kB)
Using cached widgetsnbextension-4.0.15-py3-none-any.whl (2.2 MB)
  Using cached python_dotenv-1.2.1-py3-none-any.whl.metadata (25 kB)
  Using cached pickleshare-0.7.5-py2.py3-none-any.whl.metadata (1.5 kB)
Using cached python_dotenv-1.2.1-py3-none-any.whl (21 kB)
Using cached pickleshare-0.7.5-py2.py3-none-any.whl (6.9 kB)


# Training small pre-trained ByT5 model using dataset

In [9]:
LANGS = ["grc","dan","fra","sme","deu","nav","jap","klr","eng","mul","deu_eng"]
LANGS = LANGS[-2:]
LANGS = ["mul_2","amh","pol","ote"]
LANGS = ["mul_2","amh","pol","ote"]
LANGS = ["mul-language-code","mul-language-word"]
LANGS = ["mul-language-code"]
LANGS = ["5lang-sample"]
#LANGS = ["slavic-family"]
LANGS = ["klr-20ksamples"]

PATH_JSON_DATA = "../preprocessing/preprocessing_to_json/data/"

for lang in LANGS:
    from datasets import load_dataset
    dataset = load_dataset("json", data_files={"train": PATH_JSON_DATA + lang + "_trn.json", "validation": PATH_JSON_DATA + lang.split("-")[0] + "_eval.json"})

    print(dataset["train"][0])

    from transformers import AutoTokenizer

    checkpoint = "google/byt5-small"
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    # -

    import torch

    from transformers import (
        AutoTokenizer,
        TrainingArguments,
        Trainer,
        AutoModelForSeq2SeqLM,
        Seq2SeqTrainingArguments,
        Seq2SeqTrainer
    )
    from datasets import load_dataset

    def preprocess_function(examples):
        # inputs = [prefix + doc for doc in examples["text"]]
        inputs = examples["input"]
        model_inputs = tokenizer(inputs)

        labels = tokenizer(text_target=examples["target"]) # same length irrelevant here, so no truncation or padding

        model_inputs["labels"] = labels["input_ids"]
        return model_inputs


    tokenized_dataset = dataset.map(preprocess_function, batched=True)
    print(tokenized_dataset)
    sample = tokenized_dataset["train"][0]
    print(sample["labels"])

    from transformers import DataCollatorForSeq2Seq

    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

    from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

    model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    training_args = Seq2SeqTrainingArguments(
        output_dir="byt5_small/"+lang,
        eval_strategy="epoch",
        learning_rate=5e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=4,
        predict_with_generate=True,
        # save_strategy="epoch",
        fp16=False,# did not work with True #change to bf16=True for XPU 
        #bf16 = True, # performance was best without setting fp16 or bf16 to true
        push_to_hub=True,
        warmup_steps = 500,
        # load_best_model_at_end=True, # otherwise not model with minimum loss during training, like https://huggingface.co/docs/transformers/tasks/sequence_classification
    )

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset["train"],
        eval_dataset=tokenized_dataset["validation"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        # compute_metrics=compute_metrics,
    )

    trainer.train()
    trainer.push_to_hub()


{'input': 'Inflect bekt | V;IMP;NEG;TR;ABS(2,DU);ERG(1,EXCL,DU)', 'target': 'mubekuje'}
DatasetDict({
    train: Dataset({
        features: ['input', 'target', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['input', 'target', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
})
[112, 120, 101, 104, 110, 120, 109, 104, 1]


/tmp/ipykernel_888465/884237083.py:81: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss
1,0.645000,0.285506
2,0.190800,0.132391
3,0.122300,0.107730
4,0.091800,0.105346


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

# Train with more epochs

In [14]:
LANGS = ["5lang-sample"]
#LANGS = ["slavic-family"]



PATH_JSON_DATA = "../preprocessing/preprocessing_to_json/data/"

for lang in LANGS:
    from datasets import load_dataset
    dataset = load_dataset("json", data_files={"train": PATH_JSON_DATA + lang + "_trn.json", "validation": PATH_JSON_DATA + lang + "_dev.json"})

    print(dataset["train"][0])

    from transformers import AutoTokenizer

    checkpoint = "google/byt5-small"
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    # 

    import torch

    from transformers import (
        AutoTokenizer,
        TrainingArguments,
        Trainer,
        AutoModelForSeq2SeqLM,
        Seq2SeqTrainingArguments,
        Seq2SeqTrainer
    )
    from datasets import load_dataset

    def preprocess_function(examples):
        # inputs = [prefix + doc for doc in examples["text"]]
        inputs = examples["input"]
        model_inputs = tokenizer(inputs)

        labels = tokenizer(text_target=examples["target"]) # same length irrelevant here, so no truncation or padding

        model_inputs["labels"] = labels["input_ids"]
        return model_inputs


    tokenized_dataset = dataset.map(preprocess_function, batched=True)
    print(tokenized_dataset)
    sample = tokenized_dataset["train"][0]
    print(sample["labels"])

    from transformers import DataCollatorForSeq2Seq

    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

    from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

    model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    training_args = Seq2SeqTrainingArguments(
        output_dir="byt5_small/"+lang+"_16epochs",
        eval_strategy="epoch",
        learning_rate=5e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=16,
        predict_with_generate=True,
        fp16=False,# did not work with True #change to bf16=True for XPU 
        #bf16 = True, # performance was best without setting fp16 or bf16 to true
        push_to_hub=True,
        warmup_steps = 500,
        save_strategy="epoch",
        load_best_model_at_end=True, # otherwise not model with minimum loss during training, like https://huggingface.co/docs/transformers/tasks/sequence_classification,
    )

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset["train"],
        eval_dataset=tokenized_dataset["validation"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        # compute_metrics=compute_metrics,
    )

    trainer.train()
    trainer.push_to_hub()


{'input': 'Navajo: Inflect náʼáłkad | V;IND;PFV;NOM(1,GRPL)', 'target': 'ńdaʼiilkad'}
DatasetDict({
    train: Dataset({
        features: ['input', 'target', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 10000
    })
    validation: Dataset({
        features: ['input', 'target', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
})
[200, 135, 103, 100, 205, 191, 108, 108, 111, 110, 100, 103, 1]


/tmp/ipykernel_888465/2785277653.py:76: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss
1,2.696000,0.712554
2,0.779800,0.502693
3,0.566400,0.430351
4,0.407200,0.372751
5,0.355600,0.336040
6,0.315200,0.308484
7,0.278300,0.282797
8,0.232500,0.275354
9,0.211900,0.271841
10,0.199400,0.261387


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

# Testing models
+ [ ] TODO: finetune all slavic models and test them on the specific dataset
+ [ ] TODO: test 5lang-sample model on the dataset with prefixes

In [16]:
# SETTINGS = ["grc","dan","fra","sme","deu","nav","jap","klr","eng","mul","deu_eng"] + ["mul_2","ote","pol","amh"] + ["mul-language-code","mul-language-word"]
SETTINGS = ["mul"]
SETTINGS = ["5lang-sample"]
BASE_TEST_LANGUAGES_OF_SETTING = {"slavic":["pol","ces","csb","dsb","slk"],"slavic-word":["pol","ces","csb","dsb","slk"],"slavic-family":["pol","ces","csb","dsb","slk"],"5lang-sample":["deu","nav","klr","amh","grc"],"5lang-sample_16epochs":["deu","nav","klr","amh","grc"],
                                 "sme-15ksamples":["sme"],"sme-20ksamples":["sme"],"klr-15ksamples":["klr"],"klr-20ksamples":["klr"]
                                 }
CODES = {"slavic": "","slavic-word":["-lang-word"],"slavic-family":["-family","-lang-word","-word-or-family"],"5lang-sample":["-lang-word"],"5lang-sample_16epochs":["-lang-word"],"sme-15ksamples":[""],"sme-20ksamples":[""],"klr-15ksamples":[""],"klr-20ksamples":[""]}
SETTINGS = ["slavic","slavic-word"]
SETTINGS = ["sme-15ksamples","sme-20ksamples","klr-15ksamples","klr-20ksamples","5lang-sample_16epochs"]
SETTINGS = ["5lang-sample_16epochs"]

#TEST_LANGS_OLD = ["grc","dan","fra","sme","deu","nav","jap","klr","eng"] 
#TEST_LANGS = ["grc","dan","fra","sme","deu","nav","jap","klr","eng"] + ["ote","pol","amh","csb"]
#lang5_LANGS = ["deu","nav","klr","amh","grc"]
#lang_code = ""
for lang_setting in SETTINGS:
    test_langs = BASE_TEST_LANGUAGES_OF_SETTING[lang_setting]
    #if lang_setting == "mul": LANG_CODES = ["-language-word","-language-code",""] # test with no language code/word too
    #else: LANG_CODES = [""]
    #for lang_code in LANG_CODES:
        #if lang_setting in ( "deu_eng", "mul") and lang_code == "": test_langs = TEST_LANGS_OLD
        #elif lang_setting in ["mul_2","mul"]: test_langs = TEST_LANGS # test multilingual model on all languages it was trained on + Kashubian
        #elif lang_setting == "5lang-sample": test_langs = lang5_LANGS
        #elif lang_setting == "pol": test_langs = ["pol","csb"] # test Polish model on Kashubian also
        #else: 
    for lang_code in CODES[lang_setting]:
        for test_lang in test_langs:
            import torch
            from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
            PRETRAINED_MODEL = "google/byt5-small"
            FINETUNED_MODEL ="livles/"+lang_setting
            #FINETUNED_MODEL ="./byt5_small/"+lang_setting + lang_code
            MODEL = FINETUNED_MODEL
            print(MODEL)
            tokenizer = AutoTokenizer.from_pretrained(
                FINETUNED_MODEL,
            )
            model = AutoModelForSeq2SeqLM.from_pretrained(
                FINETUNED_MODEL,
            dtype=torch.float16,
            device_map="auto"
            )
            from datasets import load_dataset
            OUT_DIR = "output_byt5_small/" + lang_setting + "/" 
            PATH_TO_JSON_TEST_FILE = "../preprocessing/preprocessing_to_json/data/" + test_lang + lang_code+ "_tst.json"
            print("test file: ",PATH_TO_JSON_TEST_FILE)
            PATH_TO_ORIG_TEST_FILE = "../2023InflectionST/part1/data/" + test_lang + ".tst"
            from pathlib import Path
            Path(OUT_DIR).mkdir(parents=True, exist_ok=True)
            with open (OUT_DIR + test_lang + lang_code + "_byt5-small.out","w") as out_file, open (PATH_TO_ORIG_TEST_FILE, "r") as test_file:
                dataset = load_dataset("json",data_files={"test":PATH_TO_JSON_TEST_FILE})
                covered_test_lines = dataset["test"]["input"]
                LEN = len(covered_test_lines)
                COUNTER = 0

                # preds = map (generate,covered_test_lines)
                for line in covered_test_lines:
                    input_ids = tokenizer(line, return_tensors="pt").to(model.device)
                    output = model.generate(
                        **input_ids,
                        max_new_tokens=50,
                        num_beams=4,
                        #early_stopping=True
                    )
                    output_string = (tokenizer.decode(output[0], skip_special_tokens=True))
                    lemma, features, ref = test_file.readline().strip().split("\t")

                    # evaluate
                    if (ref == output_string):
                        COUNTER += 1
                        print(ref,output_string)
                    else:
                        print("ref:",ref,"pred:",output_string)

                    # write
                    out_file.write(lemma + "\t" +features+ "\t" + output_string + "\n")
                    out_file.flush()

                accuracy = COUNTER / LEN
                out_file.write("accuracy:"+ str(accuracy)+"\n")
                print(accuracy)

livles/5lang-sample_16epochs
test file:  ../preprocessing/preprocessing_to_json/data/deu-lang-word_tst.json
Aikido Aikido
Aikido Aikido
Aikidos Aikidos
Aikido Aikido
Anreisen Anreisen
Anreise Anreise
Anreisen Anreisen
Anreise Anreise
Anreisen Anreisen
ref: Anreise pred: Anreises
Anreisen Anreisen
Anreise Anreise
Arsenikesser Arsenikesser
Arsenikesser Arsenikesser
Arsenikessern Arsenikessern
Arsenikesser Arsenikesser
Arsenikesser Arsenikesser
Arsenikessers Arsenikessers
Arsenikesser Arsenikesser
Arsenikesser Arsenikesser
Auslassungszeichen Auslassungszeichen
Auslassungszeichen Auslassungszeichen
Auslassungszeichen Auslassungszeichen
Auslassungszeichen Auslassungszeichen
Auslassungszeichen Auslassungszeichen
Auslassungszeichens Auslassungszeichens
Auslassungszeichen Auslassungszeichen
Auslassungszeichen Auslassungszeichen
Bergenten Bergenten
Bergente Bergente
Bergenten Bergenten
Bergente Bergente
Bergenten Bergenten
ref: Bergente pred: Bergentes
Bergenten Bergenten
Bergente Bergente
Bery

Umweltministeriums Umweltministeriums
Umweltministerien Umweltministerien
Umweltministerium Umweltministerium
Unterschenkel Unterschenkel
Unterschenkel Unterschenkel
Unterschenkeln Unterschenkeln
Unterschenkel Unterschenkel
Unterschenkel Unterschenkel
Unterschenkels Unterschenkels
Unterschenkel Unterschenkel
Unterschenkel Unterschenkel
Verschwörungstheoretiker Verschwörungstheoretiker
Verschwörungstheoretiker Verschwörungstheoretiker
Verschwörungstheoretikern Verschwörungstheoretikern
Verschwörungstheoretiker Verschwörungstheoretiker
Verschwörungstheoretiker Verschwörungstheoretiker
Verschwörungstheoretikers Verschwörungstheoretikers
Verschwörungstheoretiker Verschwörungstheoretiker
Verschwörungstheoretiker Verschwörungstheoretiker
Vollwaschmittel Vollwaschmittel
Vollwaschmittel Vollwaschmittel
Vollwaschmitteln Vollwaschmitteln
Vollwaschmittel Vollwaschmittel
Vollwaschmittel Vollwaschmittel
Vollwaschmittel Vollwaschmittel
Vollwaschmittel Vollwaschmittel
Vorbilder Vorbilder
Vorbild Vorb

impften impften
impfte impfte
impfen impfen
impfen impfen
impfe impfe
impftet impftet
impften impften
infizierend infizierend
infiziert infiziert
infizieren infizieren
infizierten infizierten
infiziertet infiziertet
infizierte infizierte
infizieren infizieren
infizieret infizieret
infizierest infizierest
infiziere infiziere
infizierte infizierte
infiziertest infiziertest
infizierte infizierte
kiekend kiekend
gekiekt gekiekt
kieken kieken
kiekt kiekt
kiekten kiekten
kiektet kiektet
kiekten kiekten
kiekte kiekte
kieken kieken
kieke kieke
kiekte kiekte
kiektest kiektest
kiekte kiekte
kipp kipp
kippe kippe
kippst kippst
kippt kippt
kipptet kipptet
kippte kippte
kippen kippen
kippe kippe
kippe kippe
kippten kippten
kippte kippte
kipptet kipptet
kipptest kipptest
geklirrt geklirrt
klirrt klirrt
klirren klirren
klirre klirre
klirrt klirrt
klirrten klirrten
klirrte klirrte
klirrten klirrten
klirren klirren
klirrest klirrest
klirrten klirrten
klirrte klirrte
kontrahierend kontrahierend
kontrahi

ref: diníłhił pred: dííłhił
ref: dadiłhił pred: dadeeshhil
ref: diłhił pred: diyííłhil
diłhił diłhił
ref: dazhdiłhił pred: dajííłhil
ref: jidiłhił pred: jííłhil
ref: jidiłhił pred: jiłhił
ref: dadiniitʼil pred: dadiil
ref: diniitʼil pred: diil
ref: dinishʼil pred: diʼil
ref: dadinohʼil pred: dadoohʼil
ref: dinohʼil pred: dooʼil
ref: diníʼil pred: díʼil
ref: dadiʼil pred: daʼil
diʼil diʼil
diʼil diʼil
ref: dazhdiʼil pred: dajizl
jidiʼil jidiʼil
jidiʼil jidiʼil
ref: dahoshiiké pred: hadeiilké
ref: hoshiiké pred: haiilké
ref: hoshishké pred: hashké
ref: dahoshohké pred: hadaahshké
ref: hoshohké pred: hashké
ref: hoshíníké pred: hashké
ref: dahashké pred: hadeishké
hashké hashké
hashké hashké
ref: dahojishké pred: hadajishké
ref: hojishké pred: hajishké
ref: hojishké pred: hajishké
ref: dahashiiké pred: hadashiilka
ref: hashiiké pred: hashke
ref: hashishké pred: hááshke
ref: dahashohké pred: hadashoozhka
ref: hashohké pred: hashoozhka
ref: hashíníké pred: hááshke
ref: dahashké pred: hadeis

ndaałtʼiʼ ndaałtʼiʼ
ndeiłtʼiʼ ndeiłtʼiʼ
neiłtʼiʼ neiłtʼiʼ
neiłtʼiʼ neiłtʼiʼ
njiłtʼiʼ njiłtʼiʼ
ref: ndasiiltʼiʼ pred: nidasiiltʼiʼ
niséłtʼiʼ niséłtʼiʼ
ndasoołtʼiʼ ndasoołtʼiʼ
nisoołtʼiʼ nisoołtʼiʼ
nisíníłtʼiʼ nisíníłtʼiʼ
ref: neistʼiʼ pred: neiłtʼiʼ
ref: njistʼiʼ pred: nijistʼiʼ
ref: njistʼiʼ pred: njiłtʼiʼ
ndadiiltʼih ndadiiltʼih
ndiiltʼih ndiiltʼih
ndeeshtʼih ndeeshtʼih
ndoołtʼih ndoołtʼih
ndííłtʼih ndííłtʼih
neidoołtʼih neidoołtʼih
nizhdoołtʼih nizhdoołtʼih
nizhdoołtʼih nizhdoołtʼih
ref: neiitʼeesh pred: niiʼeesh
ref: naashʼeesh pred: neiʼeesh
ndaahʼeesh ndaahʼeesh
naahʼeesh naahʼeesh
ref: naniʼeesh pred: naaʼeesh
neiʼeesh neiʼeesh
njiʼeesh njiʼeesh
ref: nishiitʼeezh pred: nishiiʼéézh
ref: nishéʼeezh pred: nishéʼéézh
ref: ndashooʼeezh pred: nidashooʼéézh
ref: nishooʼeezh pred: nishooʼéézh
ref: nishíníʼeezh pred: nishíníʼéézh
ref: ndeizhʼeezh pred: ndeizhʼéézh
ref: ndajizhʼeezh pred: nidajizhʼéézh
ndadiitʼish ndadiitʼish
ndeeshʼish ndeeshʼish
ndadoohʼish ndadoohʼish
ndííʼish ndííʼish


...

ሚስትዎ ሚስትዎ
ሚስቱ ሚስቱ
ref: ሚኒስትሯ pred: ሚኒስትሮ
ሚኒስትሩ ሚኒስትሩ
ሚኒስትሮቹ ሚኒስትሮቹ
ሚኒስትሮች ሚኒስትሮች
ሚኒስትሮቻችን ሚኒስትሮቻችን
ሚኒስትሮቼ ሚኒስትሮቼ
ሚኒስትሮቻችሁ ሚኒስትሮቻችሁ
ሚኒስትሮችህ ሚኒስትሮችህ
ሚኒስትሮቻቸው ሚኒስትሮቻቸው
ሚኒስትሮቹ ሚኒስትሮቹ
ref: ሚኒስትራችን pred: ሚኒስትሮቻችን
ሚኒስትራችሁ ሚኒስትራችሁ
ሚኒስትርሽ ሚኒስትርሽ
ref: ሚኒስትሯ pred: ሚኒስትሮ
ማህጸን ማህጸን
ማህጸኑ ማህጸኑ
ማህጸናችን ማህጸናችን
ref: ማህጸኔ pred: ማህጸናለሁ
ማህጸናችሁ ማህጸናችሁ
ማህጸንሽ ማህጸንሽ
ማህጸንዎ ማህጸንዎ
ማህጸንህ ማህጸንህ
ማህጸናቸው ማህጸናቸው
ref: ማህጸኗ pred: ማህጸኑ
ማህጸኑ ማህጸኑ
ref: ምራችሁ pred: ማራችሁ
ref: ምረው pred: ማርተው
እንማር እንማር
ref: ማሪ pred: ማር
ref: ትምሩ pred: ትማሩ
ref: ትምር pred: ትማር
ref: ይምር pred: ይማር
ref: እምራለሁ pred: እማራለሁ
ref: ትምራላችሁ pred: ትማራላችሁ
ref: ትምሪያለሽ pred: ትማራለሽ
ማርኩ ማርኩ
ማረ ማረ
ref: ምረናል pred: ማራችን
ref: ምረሀል pred: ማረህ
ማእበል ማእበል
ማእበሉ ማእበሉ
ref: ማእበሎቹ pred: ማእበላቸው
ማእበሎች ማእበሎች
ምርት ምርት
ምርቱ ምርቱ
ምርቶቹ ምርቶቹ
ምርቶች ምርቶች
ምርቶቼ ምርቶቼ
ምርቶቻችሁ ምርቶቻችሁ
ምርቶችዎ ምርቶችዎ
ምርቶችህ ምርቶችህ
ምርቶቻቸው ምርቶቻቸው
ምርቶቹ ምርቶቹ
ምርታችን ምርታችን
ምርትዎ ምርትዎ
ምርታቸው ምርታቸው
ምርቱ ምርቱ
ምርጥ ምርጥ
ምርጡ ምርጡ
ምርጦች ምርጦች
ሣጥን ሣጥን
ref: ሣጥኑ pred: ሣጥኗ
ሣጥኖቹ ሣጥኖቹ
ሣጥኖች ሣጥኖች
ሣጥኖቻችን ሣጥኖቻችን
ሣጥኖቼ ሣጥኖቼ
ሣጥኖቻችሁ ሣጥኖቻችሁ
ሣጥኖችሽ ሣጥኖችሽ
ሣጥኖችህ ሣ

ያክል ያክል
ref: ያክል pred: ያከል
ref: ያክላል pred: ያከላለች
ref: አከለ pred: አከል
ref: አክሏል pred: አክለው
አክብረሽ አክብረሽ
ማክበር ማክበር
እናክብር እናክብር
ላክብር ላክብር
ታክብር ታክብር
ያክብር ያክብር
ref: ያከብሩ pred: ያክብር
ref: ታከብሪያለሽ pred: ታክብራለሽ
ref: አከበረች pred: አክብረች
ref: አከበረ pred: አክብረ
አክብረናል አክብረናል
አክብራችኋል አክብራችኋል
አክብረሻል አክብረሻል
ref: አክብሯል pred: አክብረው
ref: አይቼ pred: አይታለሁ
ref: አይተሽ pred: አይታለሽ
አይተህ አይተህ
ref: አይቶ pred: አይተው
ref: እንይ pred: እንአይ
ref: ይይ pred: ያይ
ref: ታዪ pred: ታያለሽ
ታያላችሁ ታያላችሁ
ref: ታያለሽ pred: ታይሽ
ref: አየን pred: አይናል
ref: አያችሁ pred: አይታችሁ
ref: አዩ pred: አይተዋል
ref: አየች pred: አይተች
አይታለች አይታለች
አይጥ አይጥ
አይጡ አይጡ
አይጦቹ አይጦቹ
አይጦች አይጦች
ref: አድጋችሁ pred: አደጋችሁ
ref: አድገሽ pred: አደግሽ
ref: አድጋ pred: አደገች
ref: አድገህ pred: አደግተህ
ref: እደጊ pred: ታደግ
ref: ይደግ pred: ያደግ
ref: ታድጉ pred: ታደግ
ref: ያድግ pred: ያደግ
ref: አድጋለሁ pred: አደጋለሁ
አደጋችሁ አደጋችሁ
ref: አድገናል pred: አደጋችን
ref: አድጋችኋል pred: አደጋችኋል
ref: አድጋለች pred: አደገች
ref: አድጓል pred: አደግተው
ref: አጠራጥሮ pred: አጠራጥር
ref: ማጠራጠር pred: ማጠራጥር
ያጠራጥር ያጠራጥር
ያጠራጥር ያጠራጥር
ያጠራጥራል ያጠራጥራል
ref: አጠራጠረ pred: አጠራጥረ
re

ref: βρᾰχῠτήτοιν pred: βραχύταιν
ref: βρᾰχῠ́τησῐ(ν) pred: βραχύταις
ref: βρᾰχῠ́τητῐ pred: βραχῑ́τῳ
ref: βρᾰχῠτήτοιν pred: βραχύταιν
ref: βρᾰχῠτήτων pred: βραχῑ́των
ref: βρᾰχῠ́τητος pred: βραχῑ́τας
ref: βρᾰχῠ́τητε pred: βραχύτᾱ
ref: βρᾰχῠ́τητες pred: βραχύται
ref: βρᾰχῠ́της pred: βραχύτης
ref: βρᾰχῠ́τητε pred: βραχύτᾱ
ref: βρᾰχῠ́τητες pred: βραχύται
ref: βρᾰχῠ́τη pred: βραχύτης
γοεδνᾱ́ γοεδνᾱ́
γοεδνώ γοεδνώ
ref: γοεδνᾱ́ς pred: γοεδναί
γοεδνᾰ́ γοεδνᾰ́
γοεδνήν γοεδνήν
γοεδνόν γοεδνόν
γοεδνοῖν γοεδνοῖν
γοεδνοῖν γοεδνοῖν
γοεδνοῖς γοεδνοῖς
γοεδνῇ γοεδνῇ
γοεδνῷ γοεδνῷ
γοεδνῷ γοεδνῷ
γοεδνοῖν γοεδνοῖν
γοεδνοῖν γοεδνοῖν
γοεδνῶν γοεδνῶν
γοεδνῶν γοεδνῶν
γοεδνῆς γοεδνῆς
γοεδνοῦ γοεδνοῦ
γοεδνοῦ γοεδνοῦ
γοεδνᾱ́ γοεδνᾱ́
γοεδνώ γοεδνώ
γοεδναί γοεδναί
γοεδνᾰ́ γοεδνᾰ́
γοεδνή γοεδνή
γοεδνός γοεδνός
γοεδνᾱ́ γοεδνᾱ́
γοεδνώ γοεδνώ
γοεδναί γοεδναί
γοεδνοί γοεδνοί
ref: γοεδνᾰ́ pred: γοεδναί
γοεδνή γοεδνή
ref: γοεδνέ pred: γοεδνός
γοεδνόν γοεδνόν
δῐθῡμῐ́ᾱ δῐθῡμῐ́ᾱ
δῐθῡμῐ́ᾱς δῐθῡμῐ́ᾱς
δῐθῡμῐ́ᾱν δῐθῡμῐ́ᾱν
δῐθῡμῐ́

ποτήριον ποτήριον
ref: ποτηρίω pred: ποτήριω
ref: ποτήριᾰ pred: ποτήριοι
ποτήριον ποτήριον
ref: πτέρνᾱ pred: πτέρνω
πτέρνᾱς πτέρνᾱς
πτέρνην πτέρνην
πτέρναιν πτέρναιν
πτέρναις πτέρναις
ref: πτέρνῃ pred: πτέρνῳ
πτέρναιν πτέρναιν
ref: πτερνῶν pred: πτέρνων
ref: πτέρνης pred: πτέρνῆς
ref: πτέρνᾱ pred: πτέρνω
πτέρναι πτέρναι
πτέρνη πτέρνη
ref: πτέρνᾱ pred: πτέρνω
πτέρναι πτέρναι
πτέρνη πτέρνη
ref: πήγμᾰτε pred: πῆγμᾱ
ref: πήγμᾰτᾰ pred: πῆγματᾰς
ref: πῆγμᾰ pred: πῆγμαν
ref: πηγμᾰ́τοιν pred: πῆγμαιν
ref: πήγμᾰσῐ pred: πῆγματᾰς
ref: πήγμᾰτῐ pred: πῆγμῐ
ref: πηγμᾰ́τοιν pred: πῆγμαιν
ref: πηγμᾰ́των pred: πῆγμῶν
ref: πήγμᾰτος pred: πῆγματος
ref: πήγμᾰτε pred: πῆγμᾱ
ref: πήγμᾰτᾰ pred: πῆγματᾰ
ref: πῆγμᾰ pred: πῆγμα
ref: πήγμᾰτε pred: πῆγμᾱ
ref: πήγμᾰτᾰ pred: πῆγματες
ref: πῆγμᾰ pred: πῆγμα
ref: σιραίω pred: σῐρᾰ́ιω
ref: σίραιᾰ pred: σῐρᾰ́οις
ref: σίραιον pred: σῐ́ρᾰιον
ref: σιραίοιν pred: σῐρᾰ́ιοιν
ref: σιραίοις pred: σῐρᾰ́ιοις
ref: σιραίῳ pred: σῐρᾰ́ῳ
ref: σιραίοιν pred: σῐρᾰ́ιοιν
ref: σιραίων pr

ref: ἐξομολογητῐκώ pred: ἐξομολογητικώ
ref: ἐξομολογητῐκούς pred: ἐξομολογητικούς
ref: ἐξομολογητῐκᾰ́ pred: ἐξομολογητικᾰ́
ref: ἐξομολογητῐκήν pred: ἐξομολογητικήν
ref: ἐξομολογητῐκόν pred: ἐξομολογητικόν
ref: ἐξομολογητῐκοῖν pred: ἐξομολογητικοῖν
ref: ἐξομολογητῐκοῖν pred: ἐξομολογητικοῖν
ref: ἐξομολογητῐκαῖς pred: ἐξομολογητικαῖς
ref: ἐξομολογητῐκοῖς pred: ἐξομολογητικοῖς
ref: ἐξομολογητῐκοῖς pred: ἐξομολογητικοῖς
ref: ἐξομολογητῐκῇ pred: ἐξομολογητικῇ
ref: ἐξομολογητῐκῷ pred: ἐξομολογητικῷ
ref: ἐξομολογητῐκῷ pred: ἐξομολογητικῷ
ref: ἐξομολογητῐκοῖν pred: ἐξομολογητικοῖν
ref: ἐξομολογητῐκοῖν pred: ἐξομολογητικοῖν
ref: ἐξομολογητῐκῶν pred: ἐξομολογητικῶν
ref: ἐξομολογητῐκῶν pred: ἐξομολογητικῶν
ref: ἐξομολογητῐκῆς pred: ἐξομολογητικῆς
ref: ἐξομολογητῐκοῦ pred: ἐξομολογητικῆς
ref: ἐξομολογητῐκᾱ́ pred: ἐξομολογητικᾱ́
ref: ἐξομολογητῐκώ pred: ἐξομολογητικώ
ref: ἐξομολογητῐκώ pred: ἐξομολογητικώ
ref: ἐξομολογητῐκαί pred: ἐξομολογητικαί
ref: ἐξομολογητῐκᾰ́ pred: ἐξομολογητικᾰ́
ref: ἐξομολο